In [1]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [2]:
# display multiple outputs in one row
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [3]:
connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy

data = json.loads(json_util.dumps(input_data.find()))

In [4]:
data_non_norm = pd.DataFrame(data)

In [5]:
# pd.set_option('display.width', 1000)

In [6]:
#pd.set_option('display.expand_frame_repr', False)

In [7]:
#pd.set_option('display.max_columns', 500)

In [8]:
#pd.reset_option('^display.', silent=True)

In [9]:
data_non_norm[0:1]

,_id,categories,name,id,authorId,author,description,prep_time,cook_time,ready_in_time,servings,rating,rating_count,review_count,made_it_count,api_url,url,ingredients,nutritions
0,{'$oid': '5b2257a5ff20008b5a00085d'},"[{'name': 'Vegetarian'}, {'name': 'Casserole'}...",Spinach Enchiladas,59661,662842,SADONIA2,"If you like spinach and Mexican food, you'll l...",20,20,40,4,4.33559,1180,817,2267,https://apps.allrecipes.com/v1/recipes/59661,https://www.allrecipes.com/recipe/59661/spinac...,"[{'id': 16157, 'name': '10 g butter', 'grams':...","[{'name': 'Fat', 'amount': 35.95846, 'unit': '..."


In [10]:
new_df = data_non_norm[['name', 'id', 'ingredients', 'nutritions']]

In [11]:
new_df

,name,id,ingredients,nutritions
0,Spinach Enchiladas,59661,"[{'id': 16157, 'name': '10 g butter', 'grams':...","[{'name': 'Fat', 'amount': 35.95846, 'unit': '..."
1,Stuffed Peppers,16330,"[{'id': 3103, 'name': '305 g ground beef', 'gr...","[{'name': 'Fat', 'amount': 9.378057, 'unit': '..."
2,Crispy and Tender Baked Chicken Thighs,235151,"[{'id': 10536, 'name': 'cooking spray', 'grams...","[{'name': 'Fat', 'amount': 11.88371, 'unit': '..."
3,Simple Baked Chicken Breasts,240208,"[{'id': 6494, 'name': '4 skinless, boneless ch...","[{'name': 'Fat', 'amount': 9.5803, 'unit': 'g'..."
4,Delicious Egg Salad for Sandwiches,147103,"[{'id': 16317, 'name': '8 eggs', 'grams': 400....","[{'name': 'Fat', 'amount': 31.85464, 'unit': '..."
...,...,...,...,...
6299,Miso-Glazed Skirt Steak,228049,"[{'id': 18888, 'name': '45 ml red wine vinegar...","[{'name': 'Fat', 'amount': 7.365764, 'unit': '..."
6300,Slow Cooker Herbed Turkey Breast,229659,"[{'id': 6647, 'name': '3/8 (5 pound) boneless ...","[{'name': 'Fat', 'amount': 13.42641, 'unit': '..."
6301,Chef John's Chicken Satay Burger,236032,"[{'id': 2858, 'name': '35 g peanut butter', 'g...","[{'name': 'Fat', 'amount': 8.863402, 'unit': '..."
6302,Pineapple Jerk Chicken,245877,"[{'id': 1650, 'name': '125 g uncooked long-gra...","[{'name': 'Fat', 'amount': 4.347167, 'unit': '..."


In [12]:
data_non_norm

,_id,categories,name,id,authorId,author,description,prep_time,cook_time,ready_in_time,servings,rating,rating_count,review_count,made_it_count,api_url,url,ingredients,nutritions
0,{'$oid': '5b2257a5ff20008b5a00085d'},"[{'name': 'Vegetarian'}, {'name': 'Casserole'}...",Spinach Enchiladas,59661,662842,SADONIA2,"If you like spinach and Mexican food, you'll l...",20,20,40,4,4.33559,1180,817,2267,https://apps.allrecipes.com/v1/recipes/59661,https://www.allrecipes.com/recipe/59661/spinac...,"[{'id': 16157, 'name': '10 g butter', 'grams':...","[{'name': 'Fat', 'amount': 35.95846, 'unit': '..."
1,{'$oid': '5b2257acff20008b5a00085e'},"[{'name': 'Gluten Free'}, {'name': 'Dairy Free...",Stuffed Peppers,16330,25072,BDEGER,Green peppers stuffed with ground beef and ric...,20,60,80,4,4.28910,1605,1111,2687,https://apps.allrecipes.com/v1/recipes/16330,https://www.allrecipes.com/recipe/16330/stuffe...,"[{'id': 3103, 'name': '305 g ground beef', 'gr...","[{'name': 'Fat', 'amount': 9.378057, 'unit': '..."
2,{'$oid': '5b2257b0ff20008b5a00085f'},"[{'name': 'Chicken'}, {'name': 'Main'}, {'name...",Crispy and Tender Baked Chicken Thighs,235151,2032612,Nicole Burdett,A simple spice blend gives tons of flavor to t...,10,60,70,4,4.64052,1007,786,3923,https://apps.allrecipes.com/v1/recipes/235151,https://www.allrecipes.com/recipe/235151/crisp...,"[{'id': 10536, 'name': 'cooking spray', 'grams...","[{'name': 'Fat', 'amount': 11.88371, 'unit': '..."
3,{'$oid': '5b2257b4ff20008b5a000860'},"[{'name': 'Quick Easy'}, {'name': 'Chicken'}, ...",Simple Baked Chicken Breasts,240208,196935,Always Cooking Up Something,Simple basic recipe for cooking up a bunch of ...,15,25,40,4,4.60638,282,204,2116,https://apps.allrecipes.com/v1/recipes/240208,https://www.allrecipes.com/recipe/240208/simpl...,"[{'id': 6494, 'name': '4 skinless, boneless ch...","[{'name': 'Fat', 'amount': 9.5803, 'unit': 'g'..."
4,{'$oid': '5b2257bdff20008b5a000861'},"[{'name': 'Easter'}, {'name': 'Salad'}, {'name...",Delicious Egg Salad for Sandwiches,147103,2309128,wifeyluvs2cook,This is a wonderful-tasting egg salad sandwich...,10,15,35,4,4.61898,1475,1027,3046,https://apps.allrecipes.com/v1/recipes/147103,https://www.allrecipes.com/recipe/147103/delic...,"[{'id': 16317, 'name': '8 eggs', 'grams': 400....","[{'name': 'Fat', 'amount': 31.85464, 'unit': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299,{'$oid': '5b23861bff20008b5a0020f9'},"[{'name': 'BBQ'}, {'name': 'Beef'}, {'name': '...",Miso-Glazed Skirt Steak,228049,8601924,Chef John,This succulent grilled skirt steak recipe woul...,10,8,48,4,4.13889,36,28,82,https://apps.allrecipes.com/v1/recipes/228049,https://www.allrecipes.com/recipe/228049/miso-...,"[{'id': 18888, 'name': '45 ml red wine vinegar...","[{'name': 'Fat', 'amount': 7.365764, 'unit': '..."
6300,{'$oid': '5b23861dff20008b5a0020fa'},"[{'name': 'Slow'}, {'name': 'Lowcarb'}, {'name...",Slow Cooker Herbed Turkey Breast,229659,4290639,Andersonfam09,"The most amazing, flavorful, moist, turkey bre...",15,360,390,4,4.52778,36,29,70,https://apps.allrecipes.com/v1/recipes/229659,https://www.allrecipes.com/recipe/229659/slow-...,"[{'id': 6647, 'name': '3/8 (5 pound) boneless ...","[{'name': 'Fat', 'amount': 13.42641, 'unit': '..."
6301,{'$oid': '5b238622ff20008b5a0020fb'},"[{'name': 'Asian'}, {'name': 'Chicken'}, {'nam...",Chef John's Chicken Satay Burger,236032,8601924,Chef John,"I love chicken satay, that spicy meat on a sti...",15,20,35,4,4.80556,36,26,53,https://apps.allrecipes.com/v1/recipes/236032,https://www.allrecipes.com/recipe/236032/chef-...,"[{'id': 2858, 'name': '35 g peanut butter', 'g...","[{'name': 'Fat', 'amount': 8.863402, 'unit': '..."
6302,{'$oid': '5b238629ff20008b5a0020fc'},"[{'name': 'Chicken'}, {'name': 'Main'}, {'name...",Pineapple Jerk Chicken,245877,17095853,Cans Get You Cooking®,Canned pineapple is harvested and packed at it...,10,20,30,4,4.50000,36,27,152,https://apps.allrecipes.com/v1/recipes/245877,https://www.allrecipe

In [13]:
data_non_norm.drop('url', axis=1, inplace=True)

In [14]:
ing = pd.DataFrame(json_normalize(data, record_path='ingredients',
                             meta='id', record_prefix='ingredients_', errors='ignore'))


nutritions = pd.DataFrame(json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count']))


In [15]:
nutritions

,name,amount,unit,display_value,percent_daily_value,id,prep_time,rating,rating_count,ready_in_time,review_count
0,Fat,35.95846,g,36,55,59661,20,4.33559,1180,40,817
1,Calories,509.80290,kcal,510,25,59661,20,4.33559,1180,40,817
2,Cholesterol,95.27404,mg,95,32,59661,20,4.33559,1180,40,817
3,Sodium,353.60110,mg,354,14,59661,20,4.33559,1180,40,817
4,Carbohydrates,32.26605,g,32.3,10,59661,20,4.33559,1180,40,817
...,...,...,...,...,...,...,...,...,...,...,...
126075,Sugars,0.32176,g,0.3,0,244188,10,4.80556,36,1095,30
126076,Potassium,392.87760,mg,393,11,244188,10,4.80556,36,1095,30
126077,Saturated Fat,4.65290,g,4.7,23,244188,10,4.80556,36,1095,30
126078,Calories from Fat,168.17000,kcal,168,-,244188,10,4.80556,36,1095,30


In [16]:
ing

,ingredients_id,ingredients_name,ingredients_grams,ingredients_type,id
0,16157,10 g butter,11.36000,Normal,59661
1,4405,40 g sliced green onions,41.80000,Normal,59661
2,4342,"1-1/2 cloves garlic, minced",4.80000,Normal,59661
3,4520,3/4 (10 ounce) package frozen chopped spinach ...,227.20000,Normal,59661
4,16243,180 g ricotta cheese,182.40001,Normal,59661
...,...,...,...,...,...
61189,20551,1 g ground cumin,1.05000,Normal,244188
61190,16403,0.8 g dried oregano,0.75000,Normal,244188
61191,16406,1 g freshly ground black pepper,1.05000,Normal,244188
61192,6307,15 ml olive oil,13.50000,Normal,244188


In [17]:
#ingredients.loc[ingredients['id'] == 9380]

In [18]:
#ingredients.loc[ingredients['ingredients_id'] == 2972]

In [19]:
#  ------  erstellung und data cleansing - Ingredients

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
# setze multiinde auf 'id' und 'ingredients_id'
ingredients = ing.set_index(['id', 'ingredients_id'])

# filtere alle Zutaten samit ihrer rezepte id, die weniger gleich 5 mal vorkommen
ingredients_eqles_5_ing = ingredients.groupby(
    'ingredients_id').filter(lambda x: len(x) <= 5)

# droppe alle rezepte, die eine Zutate besitzen, die weniger gleich 5 Mal vorkommen
ingredients_filt = ingredients.drop(ingredients_eqles_5_ing.index.get_level_values('id').values, level=0)

# drop alls rows with ingredients_id == 0
ingredients_eqal_zero = ingredients_filt[ingredients_filt.index.get_level_values('ingredients_id') == 0]
ingredients_filt = ingredients_filt.drop(ingredients_eqal_zero.index.get_level_values('id').values, level=0)


In [20]:
ingredients_filt

ingredients_name  \
id     ingredients_id                                                      
59661  16157                                                 10 g butter   
       4405                                     40 g sliced green onions   
       4342                                  1-1/2 cloves garlic, minced   
       4520            3/4 (10 ounce) package frozen chopped spinach ...   
       16243                                        180 g ricotta cheese   
...                                                                  ...   
229659 16423                           1-3/4 sprigs fresh thyme, divided   
       20270                          3/8 white onion, chopped - divided   
       16157                               40 g butter, sliced into pats   
       4292                             80 g chopped fresh celery leaves   
       3474            1/8 (750 milliliter) bottle white wine, or mor...   

                       ingredients_grams ingredients_type  
id     ingredients_id                                      
59661  16157                    11.36000           Normal  
       4405                     41.80000           Normal  
       4342                      4.80000           Normal  
       4520                    227.20000           Normal  
       16243                   182.40001           Normal  
...                                  ...              ...  
229659 16423                     0.66667           Normal  
       20270                    36.66667           Normal  
       16157                    37.83334           Normal  
       4292                     80.00000           Normal  
       3474                    124.69144           Normal  

[36827 rows x 3 columns]

In [21]:
#  ------ Erstellung und cleansing des Nutrition Dataframes

# erstelle neue liste auf basis der bereits gefilterten rezepte aus ingredients_filt
id_overlap_mask = nutritions['id'].isin(ingredients_filt.index.get_level_values('id').values)

# erstelle datenframe auf basis der overlapliste
nutritions_filt = nutritions.loc[id_overlap_mask]

nutrition_db = nutritions_filt.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)

# remove multiindex 'amount'
nutrition_db.columns = nutrition_db.columns.droplevel(0)

# entferne alle NA
nutrition_db = nutrition_db.dropna()

# gleiche nochmals die ids der beiden dataframe nutrition und ingredients ab, da der nutrition dataframe noch NA Werte hatt
id_overlap_mask = ingredients_filt.index.get_level_values('id').isin(nutrition_db.index)
ingredients_db = ingredients_filt[id_overlap_mask]

# abgleich ob anzahl der indizes von nutrition und zutaten dataframe gleich sind
nutrition_db.index.nunique()
ingredients_db.index.get_level_values('id').nunique()
£

SyntaxError: invalid character in identifier (<ipython-input-21-2f40ddd45d7d>, line 30)

In [ ]:
recipe_db

In [ ]:
ingredients_db.loc[ingredients_db['id'] == 20881]

In [ ]:
id_list = [215014, 8669, 16700, 16354, 12720, 8652, 8887, 51283, 45954, 213742, 14595,
           219164, 16348, 143082, 8665, 11758, 223042, 236609, 8778, 65896, 24264, 11679, 141678, 9023]

#ingredients_db.reset_index(inplace=True)

recipe_db = pd.get_dummies(ingredients_db['ingredients_id']).groupby(
    ingredients_db['id']).apply(max)



In [ ]:
new_ingredients = ingredients_db
new_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index()

In [ ]:
new_ingredients.sort_values(by='ingredients_grams')

In [ ]:
new_ingredients['quotient'] = new_ingredients['ingredients_grams']/500

In [ ]:
new_ingredients.set

In [ ]:
new_ingredients.describe()

In [ ]:
new_ingredients.hist(column='ingredients_grams', bins=200)

In [ ]:
# rechne nährwerte auf 800 kcal pro rezept um
def nutrition_to_500(x):
    a = x / x
    b = a * 800
    return b

In [ ]:
nutrition_db_800 = nutrition_db.apply(nutrition_to_800, axis = 1)
#user_nutrition = nutrition_db_800[nutrition_db_800.index.isin(id_list)]

user_recipes = recipe_db[recipe_db.index.isin(id_list)]

recommenden_nut = pd.DataFrame(data=None, columns=nutrition_db_800.columns, index=None)


In [ ]:
new_nutrition = nutrition_db.copy()

In [ ]:
new_ingredients.set_index('id', inplace=True)

In [ ]:
new_ingredients

In [ ]:
new_nutrition

In [ ]:
new_nutrition1 = new_nutrition.merge(new_ingredients, how='left', on='id')


In [ ]:
new_nutrition1

In [ ]:
#new_nutrition1 = new_nutrition1.drop('ingredients_grams',1)

In [ ]:
new_nutrition1

In [ ]:
new_nutrition2 = new_nutrition1.copy()

In [ ]:
new_nutrition2.iloc[:, 0:19+1]

In [ ]:
new_nutrition2.iloc[:, 1:19+1] *= new_nutrition2['quotient']

In [ ]:
new_nutrition2.update(new_nutrition2.iloc[:, 0:19+1].mul(new_nutrition2.quotient, 0))

In [ ]:
new_nutrition2